<a href="https://colab.research.google.com/github/ctarrington/data-science-hw/blob/master/dbda/ch08-basic-models/coin_factory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [156]:
import numpy as np
import pandas as pd

from runpy import run_path

In [157]:
!rm -rf data-science-hw
!git clone https://github.com/ctarrington/data-science-hw.git

base_path = './data-science-hw/dbda/ch08-basic-models/'
Coin = run_path(base_path+'/mint.py').get('Coin')
Mint = run_path(base_path+'/mint.py').get('Mint')


Cloning into 'data-science-hw'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 120 (delta 52), reused 16 (delta 3), pack-reused 0
Receiving objects: 100% (120/120), 612.82 KiB | 9.88 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [158]:
mint = Mint(0.3, 0.1)
coins = mint.make_coins(3)

mint = Mint(0.7, 0.1)
mint.make_coins(3, coins)

In [159]:
flips = []
for ctr in range(10):
  for coin_ctr in range(len(coins)):
    coins[coin_ctr].flip(flips)

print(flips)

[[0, 0, 1], [0, 1, 0], [0, 2, 0], [1, 3, 1], [1, 4, 1], [1, 5, 1], [0, 0, 0], [0, 1, 0], [0, 2, 1], [1, 3, 1], [1, 4, 1], [1, 5, 1], [0, 0, 0], [0, 1, 0], [0, 2, 0], [1, 3, 1], [1, 4, 1], [1, 5, 1], [0, 0, 0], [0, 1, 0], [0, 2, 1], [1, 3, 1], [1, 4, 1], [1, 5, 0], [0, 0, 0], [0, 1, 0], [0, 2, 0], [1, 3, 1], [1, 4, 1], [1, 5, 1], [0, 0, 0], [0, 1, 0], [0, 2, 1], [1, 3, 1], [1, 4, 0], [1, 5, 1], [0, 0, 1], [0, 1, 1], [0, 2, 0], [1, 3, 1], [1, 4, 0], [1, 5, 1], [0, 0, 0], [0, 1, 0], [0, 2, 1], [1, 3, 1], [1, 4, 1], [1, 5, 1], [0, 0, 1], [0, 1, 1], [0, 2, 0], [1, 3, 1], [1, 4, 1], [1, 5, 1], [0, 0, 0], [0, 1, 0], [0, 2, 0], [1, 3, 1], [1, 4, 0], [1, 5, 1]]


In [160]:
pd.DataFrame(flips, columns=['mint_id', 'coin_id', 'flip'])

,mint_id,coin_id,flip
0,0,0,1
1,0,1,0
2,0,2,0
3,1,3,1
4,1,4,1
5,1,5,1
6,0,0,0
7,0,1,0
8,0,2,1
9,1,3,1
